<a href="https://colab.research.google.com/github/isoBello/VNS_HILL_CLIMBING/blob/main/VNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plotlib

In [47]:
# Tratamento de dados para o problema

df_dados = pd.read_excel("./sample_data/dados.xlsx")

quantidade_caixas = df_dados['Caixa Id'].nunique()
quantidade_itens = df_dados['Item'].nunique()

itens = df_dados['Item'].unique().tolist()

itens_distintos_por_caixa = df_dados.groupby('Caixa Id')['Item'].nunique()
pecas_por_caixa = df_dados.groupby('Caixa Id')['Peças'].sum()

df_dados['Itens Distintos'] = df_dados['Caixa Id'].map(itens_distintos_por_caixa)
df_dados['Quantidade Total Peças'] = df_dados['Caixa Id'].map(pecas_por_caixa)
df_dados['Caixa Atribuida Onda'] = False
df_dados['Onda'] = -1
df_dados.sort_values(by='Itens Distintos', ascending=False, inplace=True)
df_dados = df_dados.groupby('Caixa Id').agg(list)

def obter_valores_unicos(valores):
  return valores[0] if valores else None

df_dados['Itens Distintos'] = df_dados['Itens Distintos'].apply(obter_valores_unicos)
df_dados['Quantidade Total Peças'] = df_dados['Quantidade Total Peças'].apply(obter_valores_unicos)

#print(df_dados)

In [ ]:
# Variáveis necessárias
limite_iteracoes_sem_melhora = 10
iteracoes_sem_melhora = 0
capacidade_onda = 2000
id_onda = 0

In [ ]:
# Solução Inicial: Construtiva gulosa, seleciona as caixas com mais itens distintos e aloca em uma onda.
# Quando a onda estiver sem capacidade para mais itens, ativa nova onda
def gera_solucao_inicial():
  capacidade_atual = 0
  caixas_alocadas = 0

  while caixas_alocadas < quantidade_caixas:
    caixa_esta_alocada = df_dados.iloc[caixas_alocadas]['Caixa Atribuida Onda']
    if caixa_esta_alocada:
      id_onda += 1
      capacidade_atual = 0
      caixas_alocadas += 1
      continue

    quantidade_total_pecas = df_dados.iloc[caixas_alocadas]['Quantidade Total Peças']
    capacidade_temporaria = capacidade_atual + quantidade_total_pecas

    if quantidade_total_pecas <= capacidade_onda and capacidade_temporaria <= capacidade_onda:
      df_dados['Onda'] = id_onda
      capacidade_atual += quantidade_total_pecas
      caixas_alocadas += 1
      df_dados.loc[caixas_alocadas, 'Caixa Atribuida Onda'] = 1

    else:
      id_onda += 1
      capacidade_atual = 0

In [46]:
# Função Objetivo
def fo(solucao):
  item_pertence_onda = 0
  for indice, linha in df_dados.iterrows():


